## 💾 The data

The company stores the information you need in the following four tables. Some of the fields are anonymized to comply with privacy regulations.

#### Doctors contains information on doctors. Each row represents one doctor.
- "DoctorID" - is a unique identifier for each doctor.
- "Region" - the current geographical region of the doctor.
- "Category" - the type of doctor, either 'Specialist' or 'General Practitioner.'
- "Rank" - is an internal ranking system. It is an ordered variable: The highest level is Ambassadors, followed by Titanium Plus, Titanium, Platinum Plus, Platinum, Gold Plus, Gold, Silver Plus, and the lowest level is Silver.
- "Incidence rate"  and "R rate" - relate to the amount of re-work each doctor generates.
- "Satisfaction" - measures doctors' satisfaction with the company.
- "Experience" - relates to the doctor's experience with the company.
- "Purchases" - purchases over the last year.

#### Orders contains details on orders. Each row represents one order; a doctor can place multiple orders.
- "DoctorID" - doctor id (matches the other tables).
- "OrderID" - order identifier.
- "OrderNum" - order number.
- "Conditions A through J" - map the different settings of the devices in each order. Each order goes to an individual patient.

#### Complaints collects information on doctor complaints.
- "DoctorID" - doctor id (matches the other tables).
- "Complaint Type" - the company's classification of the complaints.
- "Qty" - number of complaints per complaint type per doctor.

#### Instructions has information on whether the doctor includes special instructions on their orders.
- "DoctorID" - doctor id (matches the other tables).
- "Instructions" - 'Yes' when the doctor includes special instructions, 'No' when they do not.

In [64]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

data_directory = ".\\data\\"

# Load each CSV file into a pandas DataFrame
complaints_df = pd.read_csv(os.path.join(data_directory, 'complaints.csv'))
doctors_df = pd.read_csv(os.path.join(data_directory, 'doctors.csv'))
instructions_df = pd.read_csv(os.path.join(data_directory, 'instructions.csv'))
orders_df = pd.read_csv(os.path.join(data_directory, 'orders.csv'))

## Pre-Process Complaints Data
- Complaints can have multiple entry for the same `DoctorID` since there can be various `Complaint Type`.
- We transform the data to wide-data and make each `Complaint Type` as a feature.
- We add a new feature `TotalComplaints` to sum up all the complaints type made.

In [65]:
# Pivot the Complaints dataframe to transform 'Complaint Type' into columns
complaints_mod = complaints_df.pivot_table(index='DoctorID', columns='Complaint Type', values='Qty', fill_value=0).reset_index()

# Rename columns to reflect the transformation
complaints_mod.columns = ['DoctorID'] + [f'Complaint Type - {col}' for col in complaints_mod.columns[1:]]

# Create total complains column.
complaint_columns = [col for col in complaints_mod.columns if 'Complaint Type' in col]
complaints_mod['ComplaintTotal'] = complaints_mod[complaint_columns].sum(axis=1)

In [66]:
print(f"ID with complaints vs Total ID: {complaints_mod.shape[0]/doctors_df.shape[0]}")

display(complaints_mod.head())


ID with complaints vs Total ID: 0.6498855835240275


,DoctorID,Complaint Type - Correct,Complaint Type - Incorrect,Complaint Type - R&R,Complaint Type - Specific,Complaint Type - Unknown,ComplaintTotal
0,AAAEAH,1.0,0.0,0.0,0.0,0.0,1.0
1,AAAHCE,1.0,2.0,0.0,0.0,1.0,4.0
2,AABDHC,0.0,1.0,0.0,0.0,0.0,1.0
3,AABGAB,3.0,0.0,0.0,0.0,1.0,4.0
4,AACCGA,0.0,1.0,0.0,0.0,0.0,1.0


## Pre-Process Orders data
- Orders can have multiple entry for the same `DoctorID` since an ID can make multiple Orders.
- So we first create a table to count the number of orders made by each `DoctorID`.
- We then create a unique spec order. For example if Spec1 = True, Spec2 = False, Spec3 = True, then the unique feature is Spec1-Spec2.
- We then transform this list of orders with unique feature into a wide-data frame with the specs as new features (so that now we will have a unique `DoctorID`).
- This are then combined witht the total orders table to create the final 'orders_mod' table.

In [67]:
# 1. Create a column 'OrderCount' to count the number of orders for each unique 'DoctorID'
orders_mod_1 = orders_df.groupby('DoctorID').size().reset_index(name='OrderCount')

In [68]:
# Fill missing values with the most frequent value in each column from 'Condition A' onwards
conditions_columns = orders_df.columns[3:]
for column in conditions_columns:
    most_frequent_value = orders_df[column].mode()[0]
    orders_df[column] = orders_df[column].fillna(most_frequent_value)

# Create 'OrderSpec' based on conditions
def create_order_spec(row):
    conditions = [f"{col[-1]}" for col in conditions_columns[:-1] if row[col] == True]
    conditions.append(row['Condition J'])
    return '-'.join(conditions)

orders_mod_2 = orders_df.copy()
orders_mod_2['OrderSpec'] = orders_mod_2.apply(create_order_spec, axis=1)
orders_mod_2 = orders_mod_2[['DoctorID', 'OrderID', 'OrderSpec']]

# Pivot to transform 'OrderSpec' into columns
orders_mod_2_pivot = orders_mod_2.pivot_table(index='DoctorID', columns='OrderSpec', aggfunc='size', fill_value=0).reset_index()

# Merge Orders_mod_1 and Orders_mod_2 on 'DoctorID'
orders_mod = orders_mod_1.merge(orders_mod_2_pivot, on='DoctorID', how='left')

C:\Users\ibrah\AppData\Local\Temp\ipykernel_8584\1331349628.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  orders_df[column] = orders_df[column].fillna(most_frequent_value)


In [69]:
print(f"ID with orders vs Total ID: {orders_mod.shape[0]/doctors_df.shape[0]}")
display(orders_mod.head())

ID with orders vs Total ID: 0.17391304347826086


,DoctorID,OrderCount,A-B-Before,A-B-C-D-Before,A-B-D-Before,A-B-H-Before,A-Before,A-C-H-Before,A-D-Before,A-D-G-Before,...,D-F-H-Before,D-G-Before,D-G-H-Before,D-H-Before,D-I-Before,G-Before,G-H-Before,G-I-Before,H-Before,I-Before
0,AAAEAH,19,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,1,2,1
1,AABDHC,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,AABGAB,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
3,AADDIG,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AAEIEG,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Combine all data into a single Master Dataframe.

In [70]:
# Check for duplicate 'DoctorID' in each dataframe
assert not doctors_df['DoctorID'].duplicated().any(), "Duplicate DoctorID found in doctors_df"
assert not complaints_mod['DoctorID'].duplicated().any(), "Duplicate DoctorID found in complaints_mod_df"
assert not instructions_df['DoctorID'].duplicated().any(), "Duplicate DoctorID found in instructions_df"
assert not orders_mod['DoctorID'].duplicated().any(), "Duplicate DoctorID found in orders_mod_df"

# Merge dataframes
master_df = doctors_df.merge(complaints_mod, on='DoctorID', how='left')
master_df = master_df.merge(instructions_df, on='DoctorID', how='left')
master_df = master_df.merge(orders_mod, on='DoctorID', how='left')

## Clean-up the Master Dataframe for missing values:
- Replace missing values in `Complaint Type` columns with value 0- since not all `DoctorID` will have this. (only 65% of ID make complaints)
- Replace missing `Instruction` column with value 'No'. (only 17.6% of ID have instructions)
- Replace missing `Orders` column with 0. (only 17.4% of ID have instructions)
- Replace missing `Rank` values. (there are two missing values)


In [71]:
# Get columns for missing values processing
complaint_columns = [col for col in master_df.columns if 'Complaint Type' in col]
orders_columns = master_df.columns[master_df.columns.get_loc('OrderCount'):]

# Replace missing values
master_df[complaint_columns] = master_df[complaint_columns].fillna(0)
master_df['ComplaintTotal'] = master_df['ComplaintTotal'].fillna(0)
master_df['Instructions'] = master_df['Instructions'].fillna('No')
master_df[orders_columns] = master_df[orders_columns].fillna(0)

# Find the most frequent value in the 'Rank' column
most_frequent_rank = master_df['Rank'].mode()[0]

# Replace missing values in the 'Rank' column with the most frequent value
master_df['Rank'] = master_df['Rank'].fillna(most_frequent_rank)

print(f"ID with orders vs Total ID: {instructions_df.shape[0]/doctors_df.shape[0]}")
display(master_df.head())

ID with orders vs Total ID: 0.17620137299771166


,DoctorID,Region,Category,Rank,Incidence rate,R rate,Satisfaction,Experience,Purchases,Complaint Type - Correct,...,D-F-H-Before,D-G-Before,D-G-H-Before,D-H-Before,D-I-Before,G-Before,G-H-Before,G-I-Before,H-Before,I-Before
0,AHDCBA,4 15,Specialist,Ambassador,49.0,0.90,53.85,1.20,49.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABHAHF,1 8 T4,General Practitioner,Ambassador,37.0,0.00,100.00,0.00,38.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FDHFJ,1 9 T4,Specialist,Ambassador,33.0,1.53,--,0.00,34.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BJJHCA,1 10 T3,Specialist,Ambassador,28.0,2.03,--,0.48,29.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FJBEA,1 14 T4,Specialist,Ambassador,23.0,0.96,76.79,0.75,24.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Encode some features to be numerical
- Replace the '--' value in `Satisfaction` column to be 0 (i.e neutral)
- Label the `Region` column into numerical
- Convert 'Yes' to 1 and 'No' to 0 in the `Instructions` column.
- Encode the Rank label based on their hierarchy :  Ambassadors >  Titanium Plus > Titanium > Platinum Plus > Platinum > Gold Plus > Gold > Silver Plus > and the lowest level Silver.

In [72]:
# Replace '--' with 0 in the 'Satisfaction' column and convert to numeric
master_df['Satisfaction'] = master_df['Satisfaction'].replace('--', 0).astype(float)

# Label encode the 'Region' column
label_encoder = LabelEncoder()
master_df['Region'] = label_encoder.fit_transform(master_df['Region'])

# Convert 'Yes' to 1 and 'No' to 0 in the 'Instructions' column
master_df['Instructions'] = master_df['Instructions'].map({'Yes': 1, 'No': 0})

# Define the ranking order
rank_order = {
    'Ambassador': 9,
    'Titanium Plus': 8,
    'Titanium': 7,
    'Platinum Plus': 6,
    'Platinum': 5,
    'Gold Plus': 4,
    'Gold': 3,
    'Silver Plus': 2,
    'Silver': 1
}

# Encode the 'Rank' column
master_df['Rank'] = master_df['Rank'].map(rank_order)